In [38]:
import rmsd
import subprocess

result = subprocess.run(
    'calculate_rmsd ./generation/oapainn/iteration0/1/reaction_p.xyz \
    ./generation/oapainn/iteration6/1/reaction_p.xyz -e',
    capture_output=True,
    text=True,
    shell=True,
)

In [39]:
float(result.stdout)

1.7091871119834747

In [40]:
import torch

In [41]:
data = torch.load('./generation/oapainn/iteration0/trajs.pth', map_location='cpu')

In [42]:
len(data['batch0'][0][0])

3

In [43]:
data['batch0'][1][0]

tensor([ 7,  6,  8,  8,  9,  9,  9,  9,  9,  9,  9, 10,  8,  8,  9,  9, 10, 10,
        10, 10])

In [44]:
from ase import Atoms
import py3Dmol
from ase import io

file_path = './generation/oapainn/iteration0/1/reaction_r.xyz'
file_path_ = './generation/oapainn/iteration0/1/reaction_p_align.xyz'

with open(file_path, 'r') as fo:
    xyz = fo.read()

'''with open(file_path_, 'r') as fo:
    xyz_ = fo.read()'''
    
view = py3Dmol.view(width=800, height=400)
view.addModel(xyz, 'xyz')
# view.addModel(xyz_, 'xyz')
view.setStyle({'sphere': {'scale': 0.35}, 'stick': {'radius': 0.20}})
view.zoomTo()
view.show()

3Dmol.js failed to load for some reason. Please check your browser console for error messages.

In [45]:
from ase import io
import py3Dmol
import time
from IPython.display import display, clear_output

# 存储所有XYZ文件的路径
xyz_files = ['./generation/oapainn/iteration0/40/reaction_r.xyz', './generation/oapainn/iteration0/40/reaction_p.xyz']  # 示例路径

# 创建3D视图
view = py3Dmol.view(width=800, height=400)

def load_xyz(file_path):
    with open(file_path, 'r') as file:
        xyz_data = file.read()
    return xyz_data

# 添加动画效果
for file_path in xyz_files:
    xyz_data = load_xyz(file_path)
    view.removeAllModels()  # 清除上一帧
    view.addModel(xyz_data, 'xyz')
    view.setStyle({'sphere': {'scale': 0.35}, 'stick': {'radius': 0.20}})
    view.zoomTo()
    display(view)  # 显示当前帧
    clear_output(wait=True)  # 清除输出以显示下一帧
    time.sleep(0.5)  # 暂停一秒，可以调整为需要的帧间隔

# 显示最后一帧
display(view)

3Dmol.js failed to load for some reason. Please check your browser console for error messages.

In [36]:
from ase import io
from ase.neb import NEB
from ase.optimize import BFGS
from tblite.ase import TBLite
from mace.calculators import mace_off

# Read initial and final states:
initial = io.read('./generation/oapainn/iteration0/0/reaction_r.xyz')
final = io.read('./generation/oapainn/iteration0/0/reaction_p_align.xyz')

# Set calculators for initial and final states optimization:
initial.calc = mace_off(model='medium', device='cpu')
final.calc = mace_off(model='medium', device='cpu')

# Optimize initial and final states:
opt_initial = BFGS(initial, trajectory='initial_opt.traj')
opt_final = BFGS(final, trajectory='final_opt.traj')
opt_initial.run(fmax=0.05)
opt_final.run(fmax=0.05)

# Make a band consisting of 5 images using the optimized structures:
images = [initial]
images += [initial.copy() for i in range(3)]
images += [final]
neb = NEB(images, climb=True)
# Interpolate linearly the positions of the three middle images:
neb.interpolate(method='idpp')

# Set calculators for NEB images:
for image in images:
    # image.calc = mace_off(model='medium', device='cpu')
    image.calc = TBLite()

# Optimize the NEB path:
optimizer = BFGS(neb, trajectory='neb.traj')
optimizer.run(fmax=0.04, steps=20)


Using MACE-OFF23 MODEL for MACECalculator with /home/xmcao/.cache/mace/MACE-OFF23_medium.model
Using float64 for MACECalculator, which is slower but more accurate. Recommended for geometry optimization.
Using MACE-OFF23 MODEL for MACECalculator with /home/xmcao/.cache/mace/MACE-OFF23_medium.model
Using float64 for MACECalculator, which is slower but more accurate. Recommended for geometry optimization.
      Step     Time          Energy         fmax
BFGS:    0 14:09:06    -7136.715863        0.3463
BFGS:    1 14:09:07    -7136.718946        0.1181
BFGS:    2 14:09:07    -7136.719159        0.0328
      Step     Time          Energy         fmax
BFGS:    0 14:09:07    -7134.426294        1.2576
BFGS:    1 14:09:07    -7134.405447        2.9835
BFGS:    2 14:09:07    -7134.439362        0.2373
BFGS:    3 14:09:07    -7134.441367        0.1927
BFGS:    4 14:09:07    -7134.443757        0.1816
BFGS:    5 14:09:07    -7134.444377        0.1048
BFGS:    6 14:09:07    -7134.445378        0.1

False

In [40]:
from ase import Atoms
import py3Dmol


# 将Atoms对象转换为XYZ格式的字符串
i = 2
xyz = ''
xyz += str(len(images[i])) + '\n'  # 第一行是原子数
xyz += 'Water molecule\n'  # 第二行是注释
for atom in images[i]:
    xyz += f'{atom.symbol} {atom.position[0]} {atom.position[1]} {atom.position[2]}\n'

# 使用py3Dmol进行可视化
view = py3Dmol.view(width=800, height=400)
view.addModel(xyz, 'xyz')
view.setStyle({'sphere': {'scale': 0.35}, 'stick': {'radius': 0.20}})
view.zoomTo()
view.show()

3Dmol.js failed to load for some reason. Please check your browser console for error messages.

# pymatgen iddp